<h3>
ΦC<sub>4</sub>H<sub>10</sub> + (6.5)(O<sub>2</sub> + 3.76 N<sub>2</sub>
) → a CO<sub>2</sub>+ b CO + d H<sub>2</sub>O + e H<sub>2</sub>+ f O<sub>2</sub>+g N<sub>2</sub>
</h3>
<hr>
P = 1 atm<br>
T = 298.15 K<br>
Φ=0.9

<h4>Equations:</h4>
Carbon Balance:
a+b=3.6<br>
b=3.6-a

Hydrogen Balance:
2*(d+e)=9<br>
d=4.5-e

Oxygen Balance: 
2(a+f)+b+d=13<br>
2(a+f)+(3.6-a)+(4.5-e)=13<br>
a+2*f-e=4.9<br>
f=2.45- <sup>a</sup>&frasl;<sub>2</sub> + <sup>e</sup>&frasl;<sub>2</sub>

Nitrogen Balance:
g=24.44

<h3>
0.9C<sub>4</sub>H<sub>10</sub> + (6.5)(O<sub>2</sub> + 3.76 N<sub>2</sub>
) → a CO<sub>2</sub>+ (3.6-a) CO + (4.5-e) H<sub>2</sub>O + e H<sub>2</sub>+ (2.45- <sup>a</sup>&frasl;<sub>2</sub> + <sup>e</sup>&frasl;<sub>2</sub>) O<sub>2</sub>+24.44 N<sub>2</sub>
</h3>

total number of moles N<sub>T</sub><br>
= a+(3.6-a)+(4.5-e)+e+(2.45- <sup>a</sup>&frasl;<sub>2</sub> + <sup>e</sup>&frasl;<sub>2</sub>)+24.44<br>

=34.99- <sup>a</sup>&frasl;<sub>2</sub> + <sup>e</sup>&frasl;<sub>2</sub>


Equations:

H<sub>2</sub>O ⇌ H<sub>2</sub> + <sup>1</sup>&frasl;<sub>2</sub> O<sub>2</sub> .............1

CO<sub>2</sub> ⇌ CO + <sup>1</sup>&frasl;<sub>2</sub> O<sub>2</sub>  ............2

In [2]:
from my_package import *

In [4]:
a,b,d,e,f,g=symbols('a b d:g',real=True) #assigning variables to the product's coefficients

In [5]:
#Elements balancing equations
#
#All variables are obtained in terms of either 'a' or 'e'  

#Carbon balance:
c_eqn=Eq(a+b,3.6)
b=(solve(c_eqn,b)[0])

#Hydrogen balance:
h_eqn=Eq(2*d+2*e,9)
d=(solve(h_eqn,d)[0])

#Oxygen balance:
o_eqn=Eq(2*a+b+d+2*f,13)
f=(solve(o_eqn,f)[0])

#Nitrogen balance:
n_eqn=Eq(2*g,6.5*3.76*2)
g=(solve(n_eqn,g)[0])

N_total=a+b+d+e+f+g

H<sub>2</sub>O ⇌ H<sub>2</sub> + <sup>1</sup>&frasl;<sub>2</sub> O<sub>2</sub> .............1

In [6]:
kp1=e*f**0.5/(d) * (1/(N_total))**(1+0.5-1) 

CO<sub>2</sub> ⇌ CO + <sup>1</sup>&frasl;<sub>2</sub> O<sub>2</sub>  ............2

In [7]:
kp2=b*(f**0.5)/(a)*(1/N_total)**(1+0.5-1)

In [8]:
#Variables to hold values for each iteration 
temps={} #will hold each temperature with its corresponding Q

composition_values={kp1:['kp1'], kp2: ['kp2'] , a:['a'] , b:['b'] , d:['d'] , e:['e'] , f:['f'] , g:['g'] }

In [10]:
data

,Kp1,Kp2,N2,O2,CO2,H2O,CO,H2
h_f,NaN,NaN,0.0,0.0,-393520.0,-241820.0,-110530.0,0.0
298,-92.2080,-103.7620,0.0,0.0,0.0,0.0,0.0,0.0
500,-52.6910,-57.6160,5912.0,6088.0,8314.0,6920.0,5929.0,5883.0
1000,-23.1630,-23.5290,21460.0,22707.0,33405.0,25978.0,21686.0,20686.0
1200,-18.1820,-17.8710,28108.0,29765.0,44484.0,34476.0,28426.0,26794.0
1400,-14.6090,-13.8420,34936.0,36966.0,55907.0,43447.0,35338.0,33062.0
1600,-11.9210,-10.8300,41903.0,44279.0,67580.0,52844.0,42384.0,39522.0
1800,-9.8260,-8.4970,48982.0,51689.0,79442.0,62609.0,49522.0,46150.0
2000,-8.1450,-6.6350,56141.0,59199.0,91450.0,72689.0,56739.0,52932.0
2100,-7.4565,-5.8775,59399.0,62653.0,96882.0,76666.0,60052.0,55840.0


In [11]:
#Assume starting Temperature=2000 K
t=2000


#Getting properties and equations at t

if t in data.index:   #making sure t doesn't need interpolation
    props=data.loc[t]

else:
    props=interpolate(t,data)
    
    
eq1=Eq(kp1,exp(props['Kp1']))
eq2=Eq(kp2,exp(props['Kp2']))

In [12]:
eq1

Eq(1.56524758424985*e*(-0.204081632653061*a + 0.204081632653061*e + 1)**0.5*(1/(-0.5*a + 0.5*e + 34.99))**0.5/(9/2 - e), 0.000290182651641186)

In [13]:
eq2

Eq(1.56524758424985*(3.6 - a)*(-0.204081632653061*a + 0.204081632653061*e + 1)**0.5*(1/(-0.5*a + 0.5*e + 34.99))**0.5/a, 0.00131357874515992)

In [20]:
props

Kp1       -8.145
Kp2       -6.635
N2     56141.000
O2     59199.000
CO2    91450.000
H2O    72689.000
CO     56739.000
H2     52932.000
Name: 2000, dtype: float64

In [14]:
solve(eq1,a)[0]

0.02*(2.72222199299485e+38*e**3 + 1.3338874910604e+39*e**2 + 1.39730125708069e+34*e - 3.24836954722633e+34)/(5.44444398598969e+36*e**2 + 4.12609259436199e+30*e - 9.28370833731447e+30)

In [19]:
solve(eq1.subs(e,0.00916763559565160))

[3.56703729707725]

In [16]:
solve(eq2.subs(a,3.45900774413300))

[-1.37333356541368]

In [ ]:
n_e=solve(eq2,e)[0] #finding e in terms of a using first equation in order to simplify  

for i in solve(eq2.subs(a,n_e)): #solves the second equation
    if i<3.6: #as more than root will be obtained, maximum value of a is used as condition  
        new_a=i
    print(i)
for i in vals.keys(): 
    vals[i].append(i.subs({a:new_a,e:n_e.subs(a,new_a)})) #updating values for this iteration

<h3>
Q = H<sub>P</sub> - H<sub>R</sub> = 0

In [49]:
n_e.subs(a,3.45900774413300)

-1.37333356541497

In [46]:
composition_values.values()

dict_values([['kp1', 0.000290182655646885], ['kp2', 0.00131357874515520], ['a', 3.56703729707882], ['b', 0.0329627029211803], ['d', 4.49083236427847], ['e', 0.00916763572152929], ['f', 0.671065169321355], ['g', 24.4400000000000]])

In [30]:
h_f_C4H10=-103850

phi=0.9

In [31]:
H_R=phi*(h_f_C4H10) + 6.5*(data.loc['h_f']['O2']+data.loc[298]['O2']) + g*(data.loc['h_f']['N2']+data.loc[298]['N2'])

H_P=(a*(data.loc['h_f']['CO2']+props['CO2']) + b*(data.loc['h_f']['CO']+props['CO']) +d*(data.loc['h_f']['H2O']+props['H2O']) +e*(data.loc['h_f']['H2']+props['H2']) + f*(data.loc['h_f']['O2']+props['O2']) + g*(data.loc['h_f']['N2']+props['N2'])).subs({a:new_a,e:n_e.subs(a,new_a)})

Q=H_P - H_R #in this equation H_ P was obtained in terms of 'a' and 'e' as well 

temps[t]=Q

In [32]:
Q

-33054.0482218196

In [19]:
temps

{2000: -333044.333431547,
 2050: -328046.067901509,
 2100: -320962.489794315,
 2150: -311013.027843821,
 2200: -297199.952982163,
 2250: -281886.497016420,
 2300: -262241.254322321,
 2350: -237377.149794111}

In [14]:
q_pos=1000000000
q_neg=-1000000000
if Q>0: #decrease Temperature
    t-=100
    q_pos=Q
else: #higher Temperature
    t+=50
    q_neg=Q

In [ ]:
counter=0

while abs(Q) > 0.1:          
    
    #Getting properties at t
    
    if t in data.index:                #making sure t doesn't need interpolation
        props=data.loc[t]

    else:
        props=interpolate(t,data)


    eq1=Eq(kp1,exp(props['Kp1']))
    eq2=Eq(kp2,exp(props['Kp2']))
    
    n_e=solve(eq2,e)[0] #finding e in terms of a using the second equation in order to simplify  

    for i in solve(eq1.subs(e,n_e)): 
        if i<3.6: 
            new_a=i
        
    for i in composition_values.keys(): 
        composition_values[i].append(i.subs({a:new_a,e:n_e.subs(a,new_a)})) #updating values for this iteration   
        
        
    #calculating new Q and adding it to its corresponding temperature
    H_P=(a*(data.loc['h_f']['CO2']+props['CO2']) + b*(data.loc['h_f']['CO']+props['CO']) +d*(data.loc['h_f']['H2O']+props['H2O']) +e*(data.loc['h_f']['H2']+props['H2']) + f*(data.loc['h_f']['O2']+props['O2']) + g*(data.loc['h_f']['N2']+props['N2'])).subs({a:new_a,e:n_e.subs(a,new_a)})
    Q=H_P - H_R
    temps[t]=Q 
    print(temps,Q)
    
    counter+=1 
    
    if Q>0 and Q<q_pos: 
        t-=100
        q_pos=Q
    elif Q<0 and Q>q_neg:
        t+=50
        q_neg=Q
    
    if q_pos!=1000000000 and q_neg!=-1000000000:
        t=(get_key(q_neg,temps))+(-q_neg)*(get_key(q_pos,temps)-get_key(q_neg,temps))/(q_pos-q_neg)

    

In [36]:
temps

{2000: -325198.821882476,
 2050: -257064.365823944,
 2100: -185446.714917026,
 2150: -104177.256162329,
 2200: -16686.4487355830,
 2250: 72981.8395083551,
 2209.30454292279: -453.355078786961,
 2209.55577793692: -12.3149309707806,
 2209.56260132993: -0.334513013047399}

In [24]:
n_val=dict()
for i in composition_values:
    n_val[composition_values[i][0]]=composition_values[i][1:]

In [33]:
asd=pd.DataFrame(n_val,index=[i for i in range(1,10)])

asd

,kp1,kp2,a,b,d,e,f,g
1,0.00131357874992275,0.00131357874516236,3.56740303464292,0.0325969653570830,4.45925379315732,0.0407462068426804,0.686671586099882,24.4400000000000
2,0.00191842642924818,0.00191842642363409,3.55311809906046,0.0468819009395438,4.44139762365631,0.0586023763436907,0.702742138641617,24.4400000000000
3,0.00280178097872872,0.00280178097920621,3.53296856448601,0.0670314355139858,4.41621070562153,0.0837892943784684,0.725410364946227,24.4400000000000
4,0.00409188309786480,0.00409188309688171,3.50488472728724,0.0951152727127647,4.38110590908123,0.118894090918766,0.757004681815765,24.4400000000000
5,0.00597602289521474,0.00597602289500452,3.46630666051079,0.133693339489210,4.33288332563283,0.167116674367173,0.800405006928192,24.4400000000000
6,0.00818870101399894,0.00818870101437347,3.42405794308597,0.175942056914034,4.28007242886702,0.219927571132976,0.847934814023505,24.4400000000000
7,0.00633679993875337,0.00633679993807665,3.45920931182746,0.140790688172544,4.32401163976626,0.175988360233739,0.808389524203141,24.4400000000000
8,0.00634683764284144,0.00634683764365943,3.45901307716268,0.140986922837322,4.32376634647517,0.176233653524829,0.808610288181076,24.4400000000000
9,0.00634711048365354,0.00634711048342118,3.45900774413300,0.140992255866995,4.32375968016006,0.176240319839943,0.808616287853470,24.4400000000000


In [49]:
asd['temps']=temps.keys()

In [57]:
print(f'adiabatic flame temperature= {list(temps.keys())[-1]}')
for i in composition_values:
    print(f'{composition_values[i][0]}= {composition_values[i][-1]}')

adiabatic flame temperature= 2209.56260132993
kp1= 0.00634711048365354
kp2= 0.00634711048342118
a= 3.45900774413300
b= 0.140992255866995
d= 4.32375968016006
e= 0.176240319839943
f= 0.808616287853470
g= 24.4400000000000
